<a href="https://colab.research.google.com/github/JohnnyPeng123/NLP-USYD/blob/master/Lab02%20-%20Johnny's%20Answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 02

## Google Drive Access Setup

In [0]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Word2Vec

In [0]:
import nltk
nltk.download('punkt')

import pprint
import re
from lxml import etree
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

from gensim.models import Word2Vec

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Downloading TED Scripts from Google Drive 
Click on left side "Files" tab and see the file is downloaded successfully.

In [0]:
id = '1B47OiEiG2Lo1jUY6hy_zMmHBxfKQuJ8-'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('ted_en-20160408.xml')  

### Data Preprocessing

In [0]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')

# Getting contents of <content> tag from the xml file
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# Removing "Sound-effect labels" using regular expression (i.e. (Audio), (Laughter))
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# Tokenising the sentence to process it by using NLTK library
sent_text=sent_tokenize(content_text)

# Removing punctuations and changing all characters to lower case
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# Tokenising each sentence to process individual word
sentences=[]
sentences=[word_tokenize(sentence) for sentence in normalized_text]

# Prints only 10 (tokenised) sentences
print(sentences[:10])

[['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new'], ['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation'], ['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing'], ['consider', 'facit'], ['i', 'm', 'actually', 'old', 'enough', 'to', 'remember', 'them'], ['facit', 'was', 'a', 'fantastic', 'company'], ['they', 'were', 'born', 'deep', 'in', 'the', 'swedish', 'forest', 'and', 'they', 'made', 'the', 'best', 'mechanical', 'calculators', 'in', 'the', 'world'], ['everybody', 'used', 'them'], ['and', 'what', 'did', 'facit', 'do', 'when', 'the', 'electronic', 'calculator', 'came', 'along'], ['they', 'continued', 'doing', 'exactly', 'the', 'same']]


### Word2Vec - Continuous Bag-Of-Words (CBOW)

In [0]:
wv_cbow_model = Word2Vec(sentences=sentences, size=100, window=5, min_count=5, workers=4, sg=0)

In [0]:
similar_words=wv_cbow_model.wv.most_similar("man")
pprint.pprint(similar_words)

[('woman', 0.8512791991233826),
 ('guy', 0.8174343109130859),
 ('lady', 0.7742480635643005),
 ('gentleman', 0.7678461670875549),
 ('boy', 0.7588591575622559),
 ('girl', 0.7372721433639526),
 ('soldier', 0.7263712882995605),
 ('kid', 0.699205219745636),
 ('photographer', 0.6811155676841736),
 ('poet', 0.6806650161743164)]


### Word2Vec - Skip Gram

In [0]:
wv_sg_model = Word2Vec(sentences=sentences, size=100, window=5, min_count=5, workers=4, sg=1)

In [0]:
similar_words=wv_sg_model.wv.most_similar("man")
pprint.pprint(similar_words)

[('woman', 0.7736088633537292),
 ('guy', 0.741553783416748),
 ('soldier', 0.7228139638900757),
 ('rabbi', 0.7062848806381226),
 ('pianist', 0.6645767092704773),
 ('boy', 0.6637154221534729),
 ('michelangelo', 0.6623799800872803),
 ('adage', 0.6590092182159424),
 ('gentleman', 0.6551796793937683),
 ('pope', 0.6545050144195557)]


## Word2Vec vs FastText

Let's try to find out the difference between Word2Vec and FastText

Word2Vec - Skipgram cannot find similar word "electrofishing" as "electrofishing" is not in the vocabulary - so you can see the error

In [0]:
similar_words=wv_sg_model.wv.most_similar("electrofishing")
pprint.pprint(similar_words)

KeyError: ignored

### FastText - Skip Gram

You can find that FastText works extremely well

In [0]:
from gensim.models import FastText

In [0]:
ft_sg_model = FastText(sentences, size=100, window=5, min_count=5, workers=4, sg=1)

In [0]:
result=ft_sg_model.wv.most_similar("electrofishing")
pprint.pprint(result)

[('electrolux', 0.7936742305755615),
 ('airbus', 0.7896213531494141),
 ('electrolyte', 0.7837408781051636),
 ('electro', 0.7774487733840942),
 ('railroads', 0.7706649899482727),
 ('electric', 0.7632849216461182),
 ('gastric', 0.7626646757125854),
 ('railroad', 0.7593973278999329),
 ('electrogram', 0.7557892799377441),
 ('fukushima', 0.748328685760498)]


### FastText - Continuous Bag-Of-Words (CBOW)

In [0]:
ft_cbow_model = FastText(sentences, size=100, window=5, min_count=5, workers=4, sg=0)

In [0]:
result=ft_cbow_model.wv.most_similar("electrofishing")
pprint.pprint(result)

[('electric', 0.9211210608482361),
 ('electro', 0.9065508842468262),
 ('electrolux', 0.8940919041633606),
 ('electron', 0.8820742964744568),
 ('electronic', 0.8793923258781433),
 ('electrical', 0.8763437271118164),
 ('electrode', 0.8752334117889404),
 ('electroshock', 0.875156044960022),
 ('electrolyte', 0.874466061592102),
 ('electromagnet', 0.8660199642181396)]


## King + Woman - Man = ?

Try both CBOW and Skip Gram model to calculate "King - Man + Woman = ?"

In [0]:
result = wv_cbow_model.wv.most_similar(positive=['king' , 'woman'], negative=['man'], topn=1)
print(result)

[('president', 0.7961100339889526)]


In [0]:
result = wv_sg_model.wv.most_similar(positive=['king' , 'woman'], negative=['man'], topn=1)
print(result)


[('luther', 0.6692537069320679)]


In [8]:
result = ft_cbow_model.wv.most_similar(positive=['king' , 'woman'], negative=['man'], topn=1)
print(result)


[('kidding', 0.8965729475021362)]


In [0]:
result = ft_sg_model.wv.most_similar(positive=['king' , 'woman'], negative=['man'], topn=1)
print(result)


[('hawking', 0.6642031073570251)]


This is not what we expected...Probably not enough data to answer as "Queen"


### Google's Pretrained Word2Vec (Google News)
[Pretrained Model Source](https://code.google.com/archive/p/word2vec/)


Let's  try with bigger sized data (Google has already trained Word2Vec with Google News data)

**Beware, this file is big (3.39GB) - long waiting!! I recommend not to train during the lab**



In [0]:
# Beware, this file is big (3.39GB) - long waiting!!
id2 = '1cOEYOQRd1VXi7ROShhqZbioCcePvgnR5'
downloaded = drive.CreateFile({'id':id2}) 
downloaded.GetContentFile('GoogleNews-vectors-negative300.bin')  

In [0]:
from gensim.models import KeyedVectors
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

filename = 'GoogleNews-vectors-negative300.bin'
gn_wv_model = KeyedVectors.load_word2vec_format(filename, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
result = gn_wv_model.most_similar(positive=['king', 'woman'], negative=['man'], topn=1)
print(result)

[('queen', 0.7118192911148071)]


# Play with Colab Form Fields 
**The Form** supports multiple types of fields, including **input fields**, **dropdown menus**.

You can edit this section by double-clicking it. 

Let's get familiar by changing the value in each input field (on the right) and checking the changes in the code (on the left) - vice versa

In [1]:
#@title Example form fields
#@markdown please put description

string = 'examples'  #@param {type: "string"}
slider_value = 119  #@param {type: "slider", min: 100, max: 200}
number = 102  #@param {type: "number"}
date = '2020-01-05'  #@param {type: "date"}
pick_me = "monday"  #@param ['monday', 'tuesday', 'wednesday', 'thursday']
select_or_input = "apples" #@param ["apples", "bananas", "oranges"] {allow-input: true}


#print the output
print("string is",string)
print('slider_value',slider_value)

string is examples
slider_value 119


# Exercise
In this exercise, you need to implement a **'Word Algebra Calculator'  interface** using Word2Vec and FastText trained by the provided TED Scripts. The interface can be built by Colab Form Fields we just learned above.

What the users can do through the interface are:


1.   Input the word formula in the text field, e.g. King - Man + Woman
2.   Select the word embedding model from dropdown menu, either Word2Vec or FastText
3.   Select the training architecture from dropdown menu, either CBOW or Skip Gram
4.   Get(print out) the resulted word of the input formula by running the form (same to running the code section)



Note: 
1. Please **do not** put the training process into your form section.
2. Please make your interface 'user-friendly' and instructional for users to use, e.g. by adding proper explaination or guide
3. We will use formula like "Word1 + Word2 + Word3 - Word4" to test your interface, the number of the words and the sign between each two words can vary.



#Start your excercise from here


## 1.The TED scripts
The code is already provided in the above sections 'Google Drive Access Setup' and 'Word2Vec'

In [2]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import nltk
nltk.download('punkt')
import pprint
import re
from lxml import etree
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import Word2Vec
from gensim.models import FastText
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1B47OiEiG2Lo1jUY6hy_zMmHBxfKQuJ8-'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('ted_en-20160408.xml') 

targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')

# Getting contents of <content> tag from the xml file
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# Removing "Sound-effect labels" using regular expression (i.e. (Audio), (Laughter))
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# Tokenising the sentence to process it by using NLTK library
sent_text=sent_tokenize(content_text)

# Removing punctuations and changing all characters to lower case
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# Tokenising each sentence to process individual word
sentences=[]
sentences=[word_tokenize(sentence) for sentence in normalized_text]

# Prints only 10 (tokenised) sentences
print(sentences[:10])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new'], ['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation'], ['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing'], ['consider', 'facit'], ['i', 'm', 'actually', 'old', 'enough', 'to', 'remember', 'them'], ['facit', 'was', 'a', 'fantastic', 'company'], ['they', 'were', 'born', 'deep', 'in', 'the', 'swedish', 'forest', 'and', 'they', 'made', 'the', 'best', 'mechanical', 'calculators', 'in', 'the', 'world'], ['everybody', 'used', 'them'], ['and', 'what', 'did', 'facit', 'do', 'when', 'the', 'electronic', 'calculator', 'came', 'along'], ['they', 'continued', 'doi

## 2.Build your word embedding models

In [0]:
## Please generate four different types of word embedding models with TED data
## The parameter for all four models:  size=100, window=5, min_count=5, workers=4.

def get_4_models(sentences):

    #put your model training code here
    wv_cbow_model = Word2Vec(sentences=sentences, size=100, window=5, min_count=5, workers=4, sg=0)
    wv_sg_model = Word2Vec(sentences=sentences, size=100, window=5, min_count=5, workers=4, sg=1)
    ft_cbow_model = FastText(sentences, size=100, window=5, min_count=5, workers=4, sg=0)
    ft_sg_model = FastText(sentences, size=100, window=5, min_count=5, workers=4, sg=1)

    return wv_cbow_model, wv_sg_model, ft_cbow_model, ft_sg_model
wv_cbow_model, wv_sg_model, ft_cbow_model, ft_sg_model = get_4_models(sentences)

##3.Build your Interface

You can edit the following form elements to build your interface

In [38]:
#@title Word Algebra Calculator 

#@markdown Please select the model and formula to calculate the word algebra

# Get the input
Formula = 'University + Students' #@param {type: "string"}
Formula = Formula.lower()
Model = 'Word2Vec' #@param ['Word2Vec', 'FastText']
Architecture = "CBOW" #@param ["CBOW", "Skip Gram"]

## 1.choose the corresponding model 
## 2.processing the formula to extract the postive and negative word list
## Tip: string.split() can split the string into a list of words seperated by white space 
## 3.calculate the formula for an similar word using the selected model

## 5.print out the most similar word after 
## we run the code cell 
Formula = Formula.split()

negative_words = []
postive_words = []

for i in range(int(((len(Formula)-1)/2)+1)):
  if i == 0:
    postive_words.append(Formula[i])
  elif Formula[i*2-1] == '-':
    negative_words.append(Formula[i*2])
  else:
    postive_words.append(Formula[i*2])

if Model == 'Word2Vec' and Architecture == "Skip Gram":
  result = wv_sg_model.wv.most_similar(positive=postive_words, negative=negative_words, topn=1)
if Model == 'Word2Vec' and Architecture == "CBOW":
  result = wv_cbow_model.wv.most_similar(positive=postive_words, negative=negative_words, topn=1)
if Model == 'FastText' and Architecture == "Skip Gram":
  result = ft_sg_model.wv.most_similar(positive=postive_words, negative=negative_words, topn=1)
if Model == 'FastText' and Architecture == "CBOW":
  result = ft_cbow_model.wv.most_similar(positive=postive_words, negative=negative_words, topn=1)

print("The result is: ",result)

The result is:  [('harvard', 0.8168218731880188)]


##Expected Example


![alt text](https://usydnlpgroup.files.wordpress.com/2020/03/image-3-1.png)





---





![alt text](https://usydnlpgroup.files.wordpress.com/2020/03/image-4-1.png)





#Extensions - Learn more details!

##Word2Vec with Tensorflow - official
If you want to implement Word2Vec with Tensorflow, here is a [sample code](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/word2vec/word2vec_basic.py) by [tensorflow](https://github.com/tensorflow).

##Word2Vec with Pytorch
Pytorch version can be found [here](https://rguigoures.github.io/word2vec_pytorch/) but this is not official
